<a href="https://colab.research.google.com/github/chaeuda/HealthBot/blob/main/Lenny_Chatbot!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Lenny Chatbot Colab!

This Colab notebook contains all of the code you need to make a basic chatbot that will answer questions about a corpus of text. Colab is a cloud-based programming environment which will let you run all of this code from your browser.

At each step, follow the written instructions and press the "play" button next to the code sample in order to run it.

**Important Note:** This is a basic chatbot running on a limited selection of articles. It's only a starting point to show you what's possible!

If you have questions, feel free to reach out to me on Twitter at [@danshipper](https://www.twitter.com/danshipper).

## 1. Download our text corpus

The first thing we need to do is download the text our chatbot is going to use as reference material for answering questions.

In the Lenny Chatbot, I used every article he's written as the text corpus. But for this public codebase, I've collected two articles from his archive that we can use as a starting point.

These are the articles I'm using:

- [What is good retention?](https://www.lennysnewsletter.com/p/what-is-good-retention-issue-29)
- [How the biggest consumer apps got their first 1,000 users
](https://www.lennysnewsletter.com/p/how-the-biggest-consumer-apps-got)

You can replace these articles with any text corpus you want, however.


In [1]:
! git clone https://github.com/EveryInc/Lenny-Newsletter-Corpus

Cloning into 'Lenny-Newsletter-Corpus'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 7
Unpacking objects: 100% (9/9), 43.31 KiB | 1.97 MiB/s, done.


# 2. Install our dependencies and define our functions

In this section we'll install GPT Index and Langchain. We'll also define the functions that we'll use later to construct our index and query it.

First, let's install our dependencies.

In [2]:
!pip install gpt-index
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.7/221.7 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.0/344.0 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.2.2
    Uninstalling tena

Now, we'll define the functions we're going to use later in order to construct our index and query it.

In [3]:
from gpt_index import SimpleDirectoryReader, GPTListIndex, readers, GPTSimpleVectorIndex, LLMPredictor, PromptHelper
from langchain import OpenAI
import sys
import os
from IPython.display import Markdown, display

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 256
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600

    # define LLM
    llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="text-davinci-003", max_tokens=num_outputs))
    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)
 
    documents = SimpleDirectoryReader(directory_path).load_data()
    
    index = GPTSimpleVectorIndex(
        documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper
    )

    index.save_to_disk('index.json')

    return index

def ask_lenny():
    index = GPTSimpleVectorIndex.load_from_disk('index.json')
    while True: 
        query = input("What do you want to ask Lenny? ")
        response = index.query(query, response_mode="compact")
        display(Markdown(f"Lenny Bot says: <b>{response.response}</b>"))

# 3. Set OpenAI API Key
In order to run this notebook you'll need an API key from OpenAI. 

If you don't have one already, you can grab one by [signing up](https://platform.openai.com/overview). Then click your account icon on the top right of the screen and select "View API Keys". Create an API key.

Then run the code below and paste it into the text input.



In [4]:
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI API key here and hit enter:")

Paste your OpenAI API key here and hit enter:


# 4. Construct Index

Now we're going to construct our index. This will take every file in the folder 'Lenny-Newsletter-Corpus', split it into chunks, and embed it with OpenAI's embeddings API.

**Important Note:** This step costs money. Running it on the text corpus we've given you by default should only cost $0.03 in total. But if you use other pieces of text be careful if they're really long.


In [5]:
construct_index('/content/Lenny-Newsletter-Corpus')

# 5. Ask Questions!

Now we'll run the "ask_lenny" function we defined above. 

This will prompt the you to input a question, and then it will find chunks of text that might answer the question, and summarize the answer from those text chunks using GPT-3.

Remember, in this public Colab file we're only using two of Lenny's articles for our corpus. So it will only answer questions from:

- [What is good retention?](https://www.lennysnewsletter.com/p/what-is-good-retention-issue-29)
- [How the biggest consumer apps got their first 1,000 users
](https://www.lennysnewsletter.com/p/how-the-biggest-consumer-apps-got)


A few sample questions you can ask:

- What is good retention for a consumer social product?

- How did DoorDash get its first users?

- How did LinkedIn get started?

Again, this step costs money. So be aware!

In [6]:
ask_lenny()

What do you want to ask Lenny? What is good retention?


Lenny Bot says: <b>
Good retention is when a user continues to use a product or service over a period of time. This can be measured in terms of user retention (at 6 months) or net revenue retention (at 12 months). Generally, for consumer social products, ~25% is considered GOOD retention, and ~45% is considered GREAT retention. For consumer transactional products, ~35% is considered GOOD retention, and ~60% is considered GREAT retention.</b>

KeyboardInterrupt: ignored